In [ ]:
from functools import reduce, partial
from glob import glob
from collections import Counter

import pandas as pd
import dill

In [ ]:
with open('./read_csv.pkl', 'rb') as f:
    read_csv = dill.load(f)

In [ ]:
# RawFiles
file_paths = glob("./data/raw_data/*.csv")
file_paths

In [ ]:
file_column_dict = dict()
for file_path in file_paths:
    df = read_csv(file_path)
    file_column_dict[file_path] = df.columns

In [ ]:
columns_list = [list(columns) for columns in file_column_dict.values()]

In [ ]:
all_columns = sum(columns_list, [])

In [ ]:
Counter(all_columns)

In [ ]:
def do_strip(col):
    return col.strip()


Counter(map(do_strip, all_columns))

In [ ]:
def composer(*funcs):
    return reduce(lambda f, g: lambda x: g(f(x)), funcs)


def do_strip(col):
    return col.strip()


def change_masan(col):
    if col == '마산':
        return '경남 마산'
    return col


preproc = composer(do_strip, change_masan)

Counter(map(preproc, all_columns))

In [ ]:
column_counter = Counter(map(preproc, all_columns))
main_cols = list(filter(lambda x: 7 <= column_counter[x], column_counter))
main_cols

In [ ]:
def composer(*funcs):
    return reduce(lambda f, g: lambda x: g(f(x)), funcs)


def do_strip(col):
    return col.strip()


def change_masan(col):
    if col == '마산':
        return '경남 마산'
    return col


def add_base(col, main_cols):
    if col in main_cols:
        return col

    exist_cols = [main_col for main_col in main_cols if main_col.endswith(col)]

    if len(exist_cols) == 1:
        return exist_cols[0]
    else:
        return col


add_base_with_main_cols = partial(add_base, main_cols=main_cols)
preproc = composer(do_strip, change_masan, add_base_with_main_cols)
Counter(map(preproc, all_columns))

In [ ]:
with open("./preproc_cols.pkl", 'wb') as f:
    dill.dump(preproc, f)